In [5]:
import spotipy
import spotipy.util as util
import numpy as np
import pandas as pd

In [6]:
absolute_path = "f:\\AI Projects\\Music Classification"
relative_path = "Data\\Spotify"
full_path = os.path.join(absolute_path, relative_path)

In [7]:
CLIENT_ID = "372d9bfe8dce48a1aac1806d4329629d"
CLIENT_SECRET = "1a6e5a340428469dba631f9ffbd4528f"

In [8]:
auth_manager = spotipy.oauth2.SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [9]:
playlists = ["4rnleEAOdmFAbRcNCgZMpY","37i9dQZF1DWWEJlAGA9gs0","37i9dQZF1DXbITWG1ZJKYt","37i9dQZF1DX1spT6G94GFC","5oM9tyhspzK18TrPrlIZro","37i9dQZF1DX55yuR78Invt"]

In [10]:
def get_playlist_tracks(playlist_id):
    playlist_data = sp.playlist_tracks(playlist_id)
    tracks = playlist_data['items']
    while playlist_data['next']:
        playlist_data = sp.next(playlist_data)
        tracks.extend(playlist_data['items'])
        
    return tracks

In [11]:
import warnings
warnings.filterwarnings('ignore')

columns = ["artist","album","track_name","track_id","released_date","added_date","genres","danceability","energy","key","loudness","mode","speechiness","acousticness","instrumentalness","liveness","valence","tempo","duration_ms","time_signature"]

metadata_playlists = pd.DataFrame(columns=columns)

for playlist in playlists:
    playlist_data = get_playlist_tracks(playlist)
    for track in playlist_data:
        artist = track["track"]["album"]["artists"][0]["name"]
        artist_uri = track["track"]["album"]["artists"][0]["uri"]
        album = track["track"]["album"]["name"]
        track_name = track["track"]["name"]
        print(track_name)
        if not (
            metadata_playlists['artist'].str.contains(artist, regex=False).any() & 
            metadata_playlists['album'].str.contains(album, regex=False).any() & 
            metadata_playlists['track_name'].str.contains(track_name, regex=False).any()):
            track_id = track["track"]["id"]
            released_date = track["track"]["album"]["release_date"]
            added_date = track["added_at"]
            genres = sp.artist(artist_uri)["genres"]
            new_row = {"artist" : artist, "album" : album, "track_name" : track_name, "track_id" : track_id,"released_date" : released_date, "added_date" : added_date,"genres" : genres}
            audio_features = sp.audio_features(track_id)[0]
            for feature in columns[7:]:
                new_row[feature] = audio_features[feature]
            metadata_playlists = metadata_playlists.append(new_row, ignore_index=True)
        else:
            print(f"Skipping: {track_name} due to a duplicate")


Every Breath You Take
Sweet Child O' Mine
Too Many Pieces
Your Love
Africa
Gucci
Livin' On A Prayer
The Final Countdown
Eye of the Tiger
Nothing's Gonna Stop Us Now
Back In Black
I Want to Know What Love Is - 1999 Remaster
Don't Stop Believin'
Patience
Another Day in Paradise - 2016 Remaster
Paradise City
Heaven
She's Like the Wind (feat. Wendy Fraser)
Welcome To The Jungle
Walk Of Life
I'm Still Standing
Don't You (Forget About Me) - Remastered 2001
I Still Haven't Found What I'm Looking For
Alone
Listen To Your Heart
You Shook Me All Night Long
Maneater
Burning Heart - From "Rocky IV" Soundtrack
Don't Dream It's Over
Never Tear Us Apart
Jump - 2015 Remaster
You Give Love A Bad Name
Carry on Wayward Son
Everytime You Go Away
Bette Davis Eyes
I Love Rock 'n Roll
Faithfully
Purple Rain
With Or Without You - Remastered
Shout
Tougher Than the Rest
Beds Are Burning - Remastered
Poison
How Am I Supposed to Live Without You
I Hate Myself for Loving You
Who Made Who
Can't Fight This Feeling
I

In [ ]:
metadata_playlists.head()

In [ ]:
metadata_playlists.to_csv(os.path.join(full_path,"metadata_1.csv"))